#Select VASCA sources based on chosen criteria

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800" #"TDS" #"CAINGSGII_10-800" #"WD" #"MDIS_10-800" #"TDS" #  _ELAISN1
region_dir_name = "./vasca_pipeline/"+region_name+"/"
region_cat_fname = region_dir_name+"region_"+region_name+"_cat.fits"

### Prepare tt_sources with variables for selection

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from vasca.region import Region
import vasca.visualization as vvis
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, get_col_cycler,add_ogrp, color_palette, nb_fig
from astropy import units as uu
from astropy.table import Column

# Get region
rc = Region()
rc.load_from_fits(region_cat_fname)
rc.tt_sources.sort(["dec"])

#Add Object group to source table
rc.copy_table_columns("tt_sources","tt_simbad",["otype","ogrp","match_distance","sp_type","main_id"],"simbad_match_id",select_matched=True)
rc.tt_sources.rename_column("match_distance","match_distance_simbad")
rc.tt_sources.rename_column("ogrp","ogrp_simbad")
sel_simbad = rc.tt_sources["sel"].data
print(f"Number of SIMBAD matches is: {sel_simbad.sum()}  ({100* sel_simbad.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_gaiadr3",["PQSO", "PGal", "PSS","match_distance","Gmag_abs","BP-RP", "ogrp", "Plx","e_Plx","Plx_dist","VarFlag","RPlx"],"gaiadr3_match_id",select_matched=True)
rc.tt_sources.rename_column("match_distance","match_distance_gaiadr3")
rc.tt_sources.rename_column("ogrp","ogrp_gaiadr3")
sel_gaiadr3 = rc.tt_sources["sel"].data
print(f"Number of GAIA matches is: {sel_gaiadr3.sum()}  ({100*sel_gaiadr3.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_gaiaedr3_wd",["Pwd"],"rg_src_id",select_matched=True)
sel_gaia2wd = rc.tt_sources["sel"].data
print(f"Number of GAIA WD matches is: {sel_gaia2wd.sum()}  ({100*sel_gaia2wd.sum() / len(rc.tt_sources)} )%")

sel_match = np.array(sel_simbad+sel_gaiadr3+sel_gaia2wd, dtype=bool)
print(f"Total number of matches is: {sel_match.sum()}  ({100*sel_match.sum() / len(rc.tt_sources)} )%")

#Copy lomb scargle result into tt_sources
rc.copy_table_columns("tt_sources","tt_lombscargle",["ls_peak_pval", "ls_model_rchiq"],"rg_src_id")

rc.tt_sources["sel"]=True
nr_srcs = rc.tt_sources['sel'].sum()
print(f"Total number of sources is: {nr_srcs}")

nr_coadd = (rc.tt_sources["coadd_src_id"]>-0.5).sum()
print(f"Total number of sources with coadd: {nr_coadd} , fraction: {nr_coadd/nr_srcs}")

display(rc.tt_sources)

#display(rc.tt_simbad[rc.tt_simbad["ogrp"]=="WD*"])
#display(rc.tt_gaiadr3)

Number of SIMBAD matches is: 2687  (63.945740123750596 )%


Number of GAIA matches is: 3302  (78.58162779628748 )%
Number of GAIA WD matches is: 11  (0.2617801047120419 )%
Total number of matches is: 3656  (87.00618752974773 )%


Total number of sources is: 4202
Total number of sources with coadd: 2453 , fraction: 0.5837696335078534


fd_src_id,nr_det,ra,dec,pos_err,pos_xv,pos_var,pos_cpval,pos_rchiq,coadd_src_id,coadd_dist,obs_filter_id,sel,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,coadd_ffactor,coadd_fdiff_s2n,rg_fd_id,rg_src_id,nr_fd_srcs,hr,hr_err,gaiaedr3_wd_match_id,simbad_match_id,gaiadr3_match_id,gfcat_src_id,gfcat_dist,flux_ne,src_name,otype,ogrp_simbad,match_distance_simbad,sp_type,main_id,PQSO,PGal,PSS,match_distance_gaiadr3,Gmag_abs,BP-RP,ogrp_gaiadr3,Plx,e_Plx,Plx_dist,VarFlag,RPlx,Pwd,ls_peak_pval,ls_model_rchiq
,,deg,deg,arcsec,arcsec2,arcsec2,,,,arcsec,,,1e-06 Jy,1e-06 Jy,,1e-12 Jy2,,,,,,,,,,,,,,arcsec,,,,,arcsec,,,,,,arcsec,,,,0.001 arcsec,0.001 arcsec,pc,,,,,
int32,int32[2],float64,float64,float32,float32,float32,float32,float32,int64,float32,int32[2],bool,float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],int64,int32,int32,float32,float32,int32,int32,int32,int64,float32,float32[2],bytes24,bytes32,bytes8,float32,bytes32,bytes32,float32,float32,float32,float32,float32,float32,bytes8,float32,float32,float32,bytes32,float32,float32,float32,float32
-1,5 .. -1,220.1390715169943,-78.32071691152815,0.22684142,-0.08022427,0.1776882,0.28521943,0.6988279,468728,0.23905067,1 .. 0,True,414.49017 .. -1.0,3.191858 .. -1.0,0.068687744 .. -100.0,11898.802 .. -1.0,0.0 .. -1.0,331.87186 .. -1.0,1.1180216 .. 1.0,13.033482 .. 0.0,-1,873876,1,-1.0,-1.0,-1,2259,2701,-1,-1.0,0.26208347 .. --,1UVA J144033.4-781914.6,RR*,Star,0.16848367,,V* V344 Aps,0.0,0.0,0.999893,0.54511434,0.45664087,0.563334,Star*,0.1894,0.0148,5279.831,VARIABLE,12.7554,-1.0,-1.0,-1.0
-1,4 .. -1,19.79593885327578,-75.06699124574038,0.48057422,1.7799766,2.8625517,0.004490181,2.3231015,1151472,0.647359,1 .. 0,True,5.786413 .. -1.0,0.43786627 .. -1.0,0.25031152 .. -100.0,9.308881 .. -1.0,4.794181e-10 .. -1.0,15.447742 .. -1.0,1.3853205 .. 1.0,3.497799 .. 0.0,-1,1818045,1,-1.0,-1.0,-1,2703,3241,-1,-1.0,0.50031143 .. --,1UVA J11911.0-750401.2,WV*,Star,0.82075334,,OGLE SMC-T2CEP-54,0.981342,3e-06,0.018655,0.8463338,-100.0,0.681774,none,-0.0055,0.1208,nan,VARIABLE,-0.0458,-1.0,-1.0,-1.0
-1,8 .. -1,21.518452116961953,-74.94033726457188,0.25355923,0.43845955,0.993351,0.01866888,1.3660051,-1,-1.0,1 .. 0,True,9.020686 .. -1.0,0.33330327 .. -1.0,0.63382846 .. -100.0,52.74124 .. -1.0,0.0 .. -1.0,43.60751 .. -1.0,-100.0 .. -100.0,-10000.0 .. -10000.0,-1,1814643,1,-1.0,-1.0,-1,2702,3240,-1,-1.0,0.79613346 .. --,1UVA J12604.4-745625.2,cC*,Star,0.5843469,,OGLE SMC-CEP-4874,0.007119,0.0,0.992839,0.611582,-100.0,0.737564,none,-0.0495,0.0649,nan,VARIABLE,-0.7622,-1.0,-1.0,-1.0
-1,21 .. 19,7.37628051678903,-72.34582444970648,0.094023064,0.07283033,0.44373125,0.0002033718,1.2652671,1251902,0.23931019,1 .. 2,True,44.67289 .. 20.190329,0.32570434 .. 0.35183197,0.020134017 .. 0.06424665,43.705227 .. 30.346352,0.0 .. 5.8510213e-33,22.47226 .. 11.176,0.91441 .. -100.0,-10.58143 .. -10000.0,-1,386837,2,0.46179053,0.0088326745,-1,-1,2274,-1,-1.0,0.14189439 .. 0.25346923,1UVA J02930.3-722045.0,none,none,-1.0,none,none,1.0,0.0,0.0,0.68663955,-100.0,0.478901,AGN,0.1025,0.1663,9756.098,NOT_AVAILABLE,0.6163,-1.0,1.4771491e-05,3.565981
-1,4 .. -1,6.476485133811305,-71.64575935955612,0.42254618,0.5118115,1.35829,0.046909314,1.4828753,-1,-1.0,1 .. 0,True,3.6794333 .. -1.0,0.3212436 .. -1.0,0.4325275 .. -100.0,6.3018556 .. -1.0,1.9788551e-10 .. -1.0,16.049904 .. -1.0,-100.0 .. -100.0,-10000.0 .. -10000.0,-1,1962250,1,-1.0,-1.0,-1,2755,3326,-1,-1.0,0.65766823 .. --,1UVA J02554.4-713844.7,RR*,Star,1.348883,,OGLE SMC706.04.008128,1.0,0.0,0.0,1.3680757,-100.0,0.515135,AGN,-0.1556,0.2602,nan,VARIABLE,-0.5981,-1.0,-1.0,-1.0
-1,3 .. -1,70.65655908743925,-53.4897268849611,0.43310454,-0.13687117,0.46440896,0.2369255,0.7200048,957734,0.3241243,1 .. 0,True,3.2716336 .. -1.0,0.33077025 .. -1.0,2.1917343 .. -100.0,23.906345 .. -1.0,4.1113073e-17 .. -1.0,37.730206 .. -1.0,1.2911669 .. 1.0,2.1189752 .. 0.0,-1,1565036,1,-1.0,-1.0,-1,-1,3087,-1,-1.0,1.4804507 .. --,1UVA J44237.6-532923.0,none,none,

### Print info for selected sources

In [3]:
# Make selection
sel_otype = rc.tt_sources["otype"] ==  "PM*" #"SN*" #"RR*"# "No*"#"HS?" #"BS*" # # "SN*" #"EB*"
sel_otypes =   (rc.tt_sources["Pwd"] >0.9) # (rc.tt_sources["otype"] ==  "WD?")  + #  + (rc.tt_sources["RPlx"]>4) (rc.tt_sources["otype"] =="WD*")*(rc.tt_sources["sp_type"] !="none")+
sel_flux_nxv = rc.tt_sources["flux_nxv"][:,0]>2
sel_flux_hr = rc.tt_sources["hr"]>2
sel_src_id = rc.tt_sources["rg_src_id"]==15179
sel_period = (rc.tt_sources["ls_peak_pval"]>-0.5) * (rc.tt_sources["ls_peak_pval"]<0.000000573303) * (rc.tt_sources["ls_model_rchiq"]<1.1)# # 3 sigma 0.002699796063, 4 sigma 0.000063342484, 5 sigma 0.000000573303
sel_srcs =  sel_otypes #sel_period #sel_otypes  #sel_src_id  #sel_flux_nxv #sel_otype # sel_flux_nxv_fuv # sel_flux_hr#

#Print oout list of source IDs to be used for further use, e.g. in inspect_sources.ipynb
print("\nsrcs_ids = [", end="")
for src_id in rc.tt_sources[sel_srcs]["rg_src_id"].data:
    print(src_id,",", end="")
print("]\n")
print("Total Nr. of sources:",sel_srcs.sum(),"\n")

#Display match and simbad table for selected sources
cols_write = ["src_name","ra","dec","ogrp_gaiadr3","otype","sp_type","main_id","nr_det","flux","flux_cpval","flux_ne","Plx_dist","ls_peak_pval","ls_model_rchiq","VarFlag","Pwd","RPlx","rg_src_id"]
display(rc.tt_sources[sel_srcs][cols_write] )#["sp_type","main_id"]
rc.tt_sources[sel_srcs][cols_write].write(region_dir_name+"sources.ecsv",overwrite=True,delimiter=',')

col_rd = ["Pwd","Plx_dist","RPlx","flux_ne"]
for col in col_rd:
    rc.tt_sources[col] = np.round(rc.tt_sources[col],2)
cols_tex = ["src_name","main_id","sp_type"]+col_rd+ ["flux_cpval"]
rc.tt_sources[sel_srcs][cols_tex].write(region_dir_name+"sources.tex", overwrite=True)



srcs_ids = [357455 ,322799 ,94932 ,87105 ,]

Total Nr. of sources: 4 



src_name,ra,dec,ogrp_gaiadr3,otype,sp_type,main_id,nr_det,flux,flux_cpval,flux_ne,Plx_dist,ls_peak_pval,ls_model_rchiq,VarFlag,Pwd,RPlx,rg_src_id
,deg,deg,,,,,,1e-06 Jy,,,pc,,,,,,
bytes24,float64,float64,bytes8,bytes32,bytes32,bytes32,int32[2],float32[2],float32[2],float32[2],float32,float32,float32,bytes32,float32,float32,int32
1UVA J04917.1-252556.5,12.321346896575122,-25.432354111852234,WD*,WD*,DA,GALEX J004917.1-252556,11 .. 1,73.65471 .. 41.309494,1.0632669e-08 .. -1.0,0.04259176 .. --,99.60556,-1.0,-1.0,NOT_AVAILABLE,1.0,39.4121,357455
1UVA J22222.9-05025.9,35.59527907918115,-0.8405203013145769,WD*,WD*,DA,GALEX J022222.7-005027,1 .. 9,26.544712 .. 10.316418,-1.0 .. 2.1149104e-07,-- .. 0.18395825,371.59525,-1.0,-1.0,NOT_AVAILABLE,0.988086,7.7587,322799
1UVA J221828.6-00012.0,334.6190008630954,-0.00334150877900888,WD*,WD*,DAH,PB 5130,7 .. 2,121.26045 .. 42.269413,2.6516346e-07 .. 0.00661471,0.04008117 .. 0.07513158,121.74485,-1.0,-1.0,NOT_AVAILABLE,0.994004,48.3832,94932
1UVA J231536.9+192449.5,348.90356888412066,19.41375937325458,WD*,WD?,,SDSS J231536.88+192448.9,15 .. 4,126.54993 .. 38.594788,7.254814e-15 .. 0.4565092,0.0423359 .. 0.055154238,168.92747,-1.0,-1.0,NOT_AVAILABLE,0.995758,39.2334,87105


In [4]:
srcs_ids = [357455 ,94932]